In [1]:
import pandas as pd
import os
import re
import pickle
from nltk import FreqDist

In [2]:
articles_metadata = pd.read_json("../data/article_metadata_with_filepaths.json")
articles_metadata.head(5)

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,[Political Communication/2011 - Bos et al. - H...,0
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,None,[Political Communication/2019 - Blassing et al...,1
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,None,None,[American Political Science Review/2017 - Cara...,2
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,None,None,[American Political Science Review/2020 - Wutt...,3
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,None,None,[Journal of European Public Policy/2014 - Trei...,4


In [3]:
articles_metadata["year"] = articles_metadata["year"].replace(3017, 2017)
articles_metadata

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,[Political Communication/2011 - Bos et al. - H...,0
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,None,[Political Communication/2019 - Blassing et al...,1
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,None,None,[American Political Science Review/2017 - Cara...,2
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,None,None,[American Political Science Review/2020 - Wutt...,3
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,None,None,[Journal of European Public Policy/2014 - Trei...,4
...,...,...,...,...,...,...,...,...,...,...,...
537,2018,Grzymala-Busse,Current History,Poland’s Path to Illiberalism,article,y,y,None,None,[Current History/2018 - Grzymala-Busse - Polan...,537
538,2017,Eichengreen,Current History,The Populist Turn in American Politics,article,y,y,None,None,[Current History/2017 - Eichengreen - The Popu...,538
539,2014,Mudde,Current History,The Far Right and the European Elections,article,y,y,None,None,[Current History/2014 - Mudde - The Far Right ...,539
540,2020,Cano,European History Quarterly,Printing the People,article,y,y,None,None,[European History Quarterly/2020 - Cano - Prin...,540


In [4]:
popid_year_dict = dict(zip(articles_metadata["pop_id"], articles_metadata["year"]))

In [5]:
# generating a new working title
articles_metadata["working_title"] = articles_metadata.apply(lambda row: row["authors"] + " " + str(row["year"]), axis=1)

In [6]:
title_freqs_dict = dict([el for el in FreqDist(articles_metadata["working_title"]).items() if el[1] > 1])
title_freqs_dict

{'Schmidt 2019': 2,
 'Rooduijn 2018': 2,
 'Micozzi 2018': 2,
 'Grzymala-Busse 2019': 4,
 'Marcos-Marne et al. 2020': 2,
 'Mudde 2014': 2,
 'Kriesi 2020': 2,
 'Rooduijn 2014': 2,
 'Jones 2019': 2}

In [7]:
letters = ["a", "b", "c", "d", "e"]
met = []
new_titles = []
for title in articles_metadata["working_title"]:
    if title in title_freqs_dict.keys():
        for n in range(title_freqs_dict[title]):
            letter = letters[n]
            title_letter = title + letter
            if title_letter not in met:
                new_titles.append(title_letter)
                met.append(title_letter)
                break
    else:
        new_titles.append(title)


In [8]:
articles_metadata["working_title"] = new_titles

In [9]:
popid_title_dict = dict(zip(articles_metadata["pop_id"], articles_metadata["working_title"]))

In [10]:
poptypes_df = pd.read_csv("../data/poptypes_df.csv")
poptypes_df["words"] = poptypes_df["words"].apply(eval)
poptypes_df

,Unnamed: 0,poptype,count,proportion_pct,words
0,0,right-wing,1967,58.195266,"[right-wing, right, rightwe, radical-right, ra..."
1,1,left-wing,582,17.218935,"[left-wing, left, leftist, radical-left, leftw..."
2,2,nationalist,214,6.331361,"[nationalism, nationalist, national, nation, n..."
3,3,nativist,73,2.159763,"[nativism, nativist]"
4,4,radical,104,3.076923,"[radical, radicalism, radically]"
5,5,moderate,42,1.242604,"[moderate, moderately]"
6,6,authoritatian,157,4.644970,"[authoritarian, authoritarianism]"
7,7,inclusive,69,2.041420,"[inclusionary, inclusive, inclusion]"
8,8,exclusive,108,3.195266,"[exclusionary, exclusion, exclusive, exclusive..."
9,9,neoliberal,64,1.893491,"[neoliberal, neo-liberal, Neoliberalism, neoli..."


In [11]:
poptypes_dict = dict(zip(poptypes_df["poptype"], poptypes_df["words"]))
poptypes_dict

{'right-wing': ['right-wing',
  'right',
  'rightwe',
  'radical-right',
  'radical-right-wing',
  'far-right',
  'rightist',
  'right-leaning',
  'radicalright',
  'rightwing',
  'extreme-right',
  'farright'],
 'left-wing': ['left-wing',
  'left',
  'leftist',
  'radical-left',
  'leftwing',
  'left-leaning',
  'leftwe',
  'leftism',
  'center-left',
  'radicalleft'],
 'nationalist': ['nationalism',
  'nationalist',
  'national',
  'nation',
  'nationalistic',
  'neo-nationalism',
  'nationalisms',
  'ethno',
  'ethno-nationalist',
  'ethnoregionalist'],
 'nativist': ['nativism', 'nativist'],
 'radical': ['radical', 'radicalism', 'radically'],
 'moderate': ['moderate', 'moderately'],
 'authoritatian': ['authoritarian', 'authoritarianism'],
 'inclusive': ['inclusionary', 'inclusive', 'inclusion'],
 'exclusive': ['exclusionary',
  'exclusion',
  'exclusive',
  'exclusively',
  'exclusionist'],
 'neoliberal': ['neoliberal',
  'neo-liberal',
  'Neoliberalism',
  'neoliberalism',
  'Neoli

In [12]:
path = "../data/large_data/populis_conc10/"
all_concs = []
for filename in os.listdir(path):
    try:
        pop_id = int(re.search("(\d+)(.txt)", filename).groups()[0])
        year = popid_year_dict[pop_id]
        title = popid_title_dict[pop_id]
        if ".txt" in filename:
            filepath = path + filename
            with open(filepath, "r") as f:
                lines = f.readlines()
            lines_splitted = [l.split() for l in lines]
            lines_splitted = [(conc, str(year), title) for conc in lines_splitted]
            all_concs.extend(lines_splitted)
    except:
        pass

In [13]:
for filename in os.listdir(path):
    pop_id = int(re.search("(\d+)(.txt)", filename).groups()[0])
    print(pop_id)

9
123
137
484
490
447
321
335
453
309
296
282
533
255
241
527
73
269
67
66
72
268
240
526
532
254
283
297
308
334
452
446
320
491
485
136
122
8
134
120
108
493
487
450
336
322
444
478
281
295
58
524
242
256
518
64
70
71
65
519
257
531
525
59
243
294
280
479
323
445
451
337
486
492
109
121
135
119
131
125
496
482
469
333
455
441
327
284
290
61
509
75
247
521
49
535
253
534
48
252
246
520
74
508
60
291
285
440
326
332
454
468
483
497
124
130
118
126
132
481
495
318
324
442
456
330
293
89
287
76
62
278
250
536
522
244
523
245
251
537
63
279
77
286
292
88
457
331
325
443
319
494
480
133
127
183
197
140
154
168
381
395
342
424
430
356
418
236
38
222
10
11
39
223
237
419
431
357
343
425
394
380
169
155
141
196
182
194
180
157
143
396
382
355
433
427
341
369
221
235
13
209
12
208
234
220
368
426
340
354
432
383
397
142
156
181
195
191
185
152
146
393
387
378
436
350
344
422
218
16
224
230
231
225
17
219
345
423
437
351
379
386
392
147
153
184
190
186
192
179
145
151
384
390
409
421
347
353
43

In [14]:
FreqDist([el[1] for el in all_concs]).most_common()


[('2020', 9264),
 ('2019', 8940),
 ('2018', 5936),
 ('2017', 5387),
 ('2016', 3690),
 ('2014', 2332),
 ('2015', 1550),
 ('2011', 1117),
 ('2013', 1088),
 ('2012', 715)]

In [15]:
print(all_concs[:10])

[(['representation', 'important', 'electoral', 'ramiﬁcation', 'european', 'democracy', 'keyword_European_Union', 'political', 'party', 'representation', 'populism', 'research', 'citizen', 'satisfaction', 'diﬀerentiate', 'inputoriente', 'procedural', 'aspect', 'politic', 'output-oriented', 'measurement'], '2020', 'Bakker et al. 2020'), (['immigration', 'policy', 'key', 'component', 'socio-cultural', 'dimension', 'european', 'democracy', 'Rovny_and_Polk_2019', 'particularly', 'populist', 'radical', 'right', 'party', 'Ivarsﬂaten', 'scholar', 'highlight', 'immigration', 'european', 'integration', 'particular'], '2020', 'Bakker et al. 2020'), (['express', 'cynicism', 'resentment', 'hatred', 'democratic', 'institution', 'govern', 'elite', 'fuel', 'support', 'populism', '.', '2019', 'highlight', 'mainstream', 'party', 'left', 'right', 'see', 'vote', 'share'], '2020', 'Bakker et al. 2020'), (['national', 'legislative', 'election', 'substantial', 'body', 'research', 'document', 'connection', 'p

In [16]:
middle = int((len(all_concs[0][0])) / 2)
middle

10

In [17]:
all_concs[0][0][middle]

'populism'

In [18]:
bool(set(["a", "b"]) & set(["c", "d"]))

False

In [19]:
len(all_concs)

40019

In [20]:
all_concs

[(['representation',
   'important',
   'electoral',
   'ramiﬁcation',
   'european',
   'democracy',
   'keyword_European_Union',
   'political',
   'party',
   'representation',
   'populism',
   'research',
   'citizen',
   'satisfaction',
   'diﬀerentiate',
   'inputoriente',
   'procedural',
   'aspect',
   'politic',
   'output-oriented',
   'measurement'],
  '2020',
  'Bakker et al. 2020'),
 (['immigration',
   'policy',
   'key',
   'component',
   'socio-cultural',
   'dimension',
   'european',
   'democracy',
   'Rovny_and_Polk_2019',
   'particularly',
   'populist',
   'radical',
   'right',
   'party',
   'Ivarsﬂaten',
   'scholar',
   'highlight',
   'immigration',
   'european',
   'integration',
   'particular'],
  '2020',
  'Bakker et al. 2020'),
 (['express',
   'cynicism',
   'resentment',
   'hatred',
   'democratic',
   'institution',
   'govern',
   'elite',
   'fuel',
   'support',
   'populism',
   '.',
   '2019',
   'highlight',
   'mainstream',
   'party',
  

In [22]:
concs_covered_types = []
for conc, year, title in all_concs:
    covered_types = []
    for key in poptypes_dict.keys():
        if set(poptypes_dict[key]) & set(conc):
            covered_types.append(key)
            conc = [w for w in conc if w not in poptypes_dict[key]]
    concs_covered_types.append((conc, covered_types, year, title))

In [28]:
concs_covered_types = []
for conc, year, title in all_concs:
    covered_types = []
    if len(conc) == 21:
        print(conc[9])
    #for key in poptypes_dict.keys():
    #    if set(poptypes_dict[key]) & set(conc):
    #        covered_types.append(key)
    #        conc = [w for w in conc if w not in poptypes_dict[key]]
    #concs_covered_types.append((conc, covered_types, year, title))

representation
particularly
support
discontent
anti-establishment
measure
exist
agreement
right
deﬁnition
aspect
vote
resistant
action
campaign
party
unashamedly
classical
market
analysis
overestimate
vulnerability
setting
U.S.
rise
Mounk_2018
tendency
2013
contemporary
dictatorship
vulnerable
case
upsurge
Eastern_Europe
instance
impact
show
opening
give
absent
precondition
condition
hinder
precondition
beneﬁts
contrast
fail
vulnerable
opportunity
coincide
suggest
immunity
elect
windfall
condition
case
assess
instance
explain
condition
highlight
range
likelihood
Trump
democracy
concern
nature
impact
deﬁnes
people
predominance
wonder
support
reason
recent_decade
cf
Institutional_Constraints
constraint
promote
ﬁxed
setup
sneaky
allow
experience
parliament
Magyar
turkish
include
include
hobble
slovak
right
’s
-
minimize
position
obstacle
contrast
tactic
tailor-made
permit
new
people
forward
step
death
latin_american
politician
turn
neoliberal
numerous
authoritarianism
country
bolivarian
r

In [23]:
concs_covered_types[:10]

[(['representation',
   'important',
   'electoral',
   'ramiﬁcation',
   'european',
   'democracy',
   'keyword_European_Union',
   'political',
   'party',
   'representation',
   'populism',
   'research',
   'citizen',
   'satisfaction',
   'diﬀerentiate',
   'inputoriente',
   'procedural',
   'aspect',
   'politic',
   'output-oriented',
   'measurement'],
  [],
  '2020',
  'Bakker et al. 2020'),
 (['immigration',
   'policy',
   'key',
   'component',
   'socio-cultural',
   'dimension',
   'european',
   'democracy',
   'Rovny_and_Polk_2019',
   'particularly',
   'populist',
   'party',
   'Ivarsﬂaten',
   'scholar',
   'highlight',
   'immigration',
   'european',
   'integration',
   'particular'],
  ['right-wing', 'radical'],
  '2020',
  'Bakker et al. 2020'),
 (['express',
   'cynicism',
   'resentment',
   'hatred',
   'democratic',
   'institution',
   'govern',
   'elite',
   'fuel',
   'support',
   'populism',
   '.',
   '2019',
   'highlight',
   'mainstream',
   'p

In [49]:
concs_covered_types[:10]

[(['representation',
   'important',
   'electoral',
   'ramiﬁcation',
   'european',
   'democracy',
   'keyword_European_Union',
   'political',
   'party',
   'representation',
   'populism',
   'research',
   'citizen',
   'satisfaction',
   'diﬀerentiate',
   'inputoriente',
   'procedural',
   'aspect',
   'politic',
   'output-oriented',
   'measurement'],
  [],
  '2020',
  'Bakker et al. 2020'),
 (['immigration',
   'policy',
   'key',
   'component',
   'socio-cultural',
   'dimension',
   'european',
   'democracy',
   'Rovny_and_Polk_2019',
   'particularly',
   'populist',
   'party',
   'Ivarsﬂaten',
   'scholar',
   'highlight',
   'immigration',
   'european',
   'integration',
   'particular'],
  ['right-wing', 'radical'],
  '2020',
  'Bakker et al. 2020'),
 (['express',
   'cynicism',
   'resentment',
   'hatred',
   'democratic',
   'institution',
   'govern',
   'elite',
   'fuel',
   'support',
   'populism',
   '.',
   '2019',
   'highlight',
   'mainstream',
   'p

In [50]:
FreqDist([el[1] for el in all_concs]).most_common()

[('2020', 9264),
 ('2019', 8940),
 ('2018', 5936),
 ('2017', 5387),
 ('2016', 3690),
 ('2014', 2332),
 ('2015', 1550),
 ('2011', 1117),
 ('2013', 1088),
 ('2012', 715)]

In [51]:
len([el for el in concs_covered_types if bool(el[1])])

11535

In [52]:
len([el for el in concs_covered_types if len(el[1])==1])

7380

In [55]:
concs_labeled = [(el[0], el[1][0], el[2], el[3]) for el in concs_covered_types if len(el[1])==1]
len(concs_labeled)

7380

In [56]:
concs_labeled[:5]

[(['legislative',
   'election',
   'substantial',
   'body',
   'research',
   'document',
   'connection',
   'political',
   'discontent',
   'populist',
   'voting',
   '.',
   '2002',
   'Norris_2005',
   'Schumacher',
   'Rooduijn',
   'expectation',
   'group',
   'relatively',
   'engaged'],
  'nationalist',
  '2020',
  'Bakker et al. 2020'),
 (['corrosive',
   'tactic',
   'exaggerated',
   'shock',
   'fascism',
   'rise',
   'the_interwar_year',
   'prominent',
   'recent',
   'case',
   'populist',
   'move',
   'burgeon',
   'literature',
   'threat',
   'U.S.',
   'democracy',
   'overestimate',
   'openness',
   'institution'],
  'authoritatian',
  '2020',
  'Weyland 2020'),
 (['recent',
   'upsurge',
   'populism',
   'fact',
   'weak',
   'institutional',
   'setting',
   'contemporary',
   'Latin_America',
   'Eastern_Europe',
   'populist',
   'leader',
   'fail',
   'machination',
   'observer',
   'overly',
   'impressed',
   'scare',
   'relatively',
   'case'],
 

In [57]:
with open("../data/concs_labeled.pickle", "wb") as f:
    pickle.dump(concs_labeled, f)